# Data Cleaning
Data cleaning is a crucial step in the data analysis process. It involves identifying and correcting (or removing) errors and inconsistencies in the data to improve its quality. Clean data leads to more accurate analyses and better decision-making. In this notebook, we will perform various data cleaning tasks on our dataset to prepare it for further analysis and modeling.

Our notebook will be structured in a way where we work per column of our dataset. Some columns will not be affected, we have written about this in our accountability report. 

## 0. Loading in dataset

In [115]:
from langdetect import detect, LangDetectException
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pathlib import Path
import pandas as pd
import numpy as np
import string 
import ast
import re 

file_path = Path("../Data/Raw/Uitgebreide_VKM_dataset.csv")
dataset = pd.read_csv(file_path, low_memory=False)

## 0.1 NLP-Function
Here we will write a function that performs most of the NLP actions. 

In [116]:
def NLP(text):

    # Returning if not a string.
    if not isinstance(text, str):
        return text 
    
    # All text to lower case --> Consistency
    text = text.lower()

    # Removing all numbers
    text = re.sub(r'\d+', '', text)

    # Removing all punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Stop words removal
    # Loading stopwords
    stopwords_en = set(stopwords.words("english"))
    stopwords_nl = set(stopwords.words("dutch"))
    stopwords_both = stopwords_en.union(stopwords_nl)

    # Language detection (try with fallback)
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "fallback"

    words = text.split()

    # Stopword removal
    if lang == "nl":
        cleaned = [w for w in words if w not in stopwords_nl]
    elif lang == "en":
        cleaned = [w for w in words if w not in stopwords_en]
    else:
        # Fallback, using combined stopwords library if unable to detect language, could result in loss of some meaning...
        cleaned = [w for w in words if w not in stopwords_both]

    # Stemming and Lemmatization
    stem_en = PorterStemmer()
    lemmatizer_en = WordNetLemmatizer()
    stem_nl = SnowballStemmer("dutch")

    cleaned_words = []
    for w in cleaned:
        if lang == "nl":
            # For Dutch we stem only because there is no Dutch lemmatizer available
            w = stem_nl.stem(w)
        elif lang == "en":
            # For English we lemmatize first then stem the word (Normal pipeline)
            w = lemmatizer_en.lemmatize(w)
            w = stem_en.stem(w)
        else:
            # As Fallback we use the English pipeline
            w = lemmatizer_en.lemmatize(w)
            w = stem_en.stem(w)

        cleaned_words.append(w)

    return " ".join(cleaned_words)



## 1. name
We will use NLP (Natural Language Processing) on this column. The column is of type string.

In [117]:
# Applying the NLP function to the 'name' column
dataset["name"] = dataset["name"].apply(NLP)

# Check to see if both languages are processed correctly 
dataset["name"].head(15)

0                                  kennismak psychologi
1                                     learn work abroad
2                                   proactiev zorgplann
3                                          rouw verlies
4                                    acuut complex zorg
5                                      kraam kind jeugd
6     profiel specifiek modul welzijn samenlev jeugd...
7                     happi citi happi peopl design lab
8                            build sustain vital commun
9                                             ggz agoog
10                                stevig stan jeugdzorg
11                                   werk gedwong kader
12                             technologie zorg welzijn
13                                        zorg dichtbij
14                                    langer thuis wijk
Name: name, dtype: object

## 2. shortdescription
This whole column will be dropped due to it's similarity with the column "module_tags". We chose to drop this column instead of the module_tags column due to the fact that in places where shortdescription holds on data, module_tags does.

In [118]:
dataset.drop(columns=["shortdescription"], inplace=True)
dataset.head()

,id,name,description,content,studycredit,location,contact_id,level,learningoutcomes,Rood,Groen,Blauw,Geel,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,4.0,2.0,1.0,5.0,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,learn work abroad,Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,5.0,3.0,1.0,1.0,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,proactiev zorgplann,Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,NaN,NaN,NaN,NaN,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,rouw verlies,In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,acuut complex zorg,In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19


## 3. content
This column is in all rows, except 13 of them, a copy of the description column's data. For the 13 exceptions we will add the data to the description column and then drop the content column after. 

In [119]:
# Masking where content and description differ
mask_diff = dataset["content"] != dataset["description"]

# Appending content data to description if they differ from eachother. 
def _merge_desc_content(row):
    desc = row["description"]
    cont = row["content"]
    if pd.isna(cont):
        return desc  # Do nothing
    if desc == cont:
        return desc # Dont append if the columns match
    
    # Appending content to description
    return str(desc) + " " + str(cont)

# Merging at places where content and description differ
dataset.loc[mask_diff, "description"] = dataset.loc[mask_diff].apply(
    _merge_desc_content, axis=1
)

# Showing all rows that changed, only their new description and their id value
updated_rows = dataset.loc[mask_diff, ["description", "id"]]
display(updated_rows.values)

# Content data can be dropped after appended to description column
dataset.drop(columns=["content"], inplace=True)

# Single row check
dataset[dataset['id'] == 179].values

array([['Deze verdiepende module gaat in op uiteenlopende aspecten die je binnen de oncologische zorg. We zorgen voor veel inbreng vanuit de dagelijkse praktijk door inzet van gastdocenten en excursies. Na deze module heb je uitgebreide inhoudelijke kennis over oncologische ziektebeelden, kun je klinisch redeneren bij kanker. Daarnaast leer je over de psychosociale gevolgen van de ziekte en wat voor begeleiding dit vraagt van zorgvragers en hun naasten. Oncologie',
        178],
       ['Ontwikkel een diepgaand begrip van klinisch redeneren in onze HBO-Verpleegkunde module Pro-Active Nursing. Leer anticiperen op zorgbehoeften door doordachte analyse en gezamenlijke, eenduidige klinische besluitvorming. Kom tot een professionele beoordeling van hoe het gaat met de zorgvrager en maak de klinische toestand inzichtelijk. Pro-active nursing',
        179],
       ['Maak impact daar waar het meest nodig is, met een focus op vroegtijdige signalering en begeleiding van jongeren met actuele pro

array([[179, 'proactiv nurs',
        'Ontwikkel een diepgaand begrip van klinisch redeneren in onze HBO-Verpleegkunde module Pro-Active Nursing. Leer anticiperen op zorgbehoeften door doordachte analyse en gezamenlijke, eenduidige klinische besluitvorming. Kom tot een professionele beoordeling van hoe het gaat met de zorgvrager en maak de klinische toestand inzichtelijk. Pro-active nursing',
        15, 'Breda', 63, 'NLQF5', 'Ntb', nan, nan, nan, nan, "['ntb']",
        0.7, 185, 2, 42, '2025-10-02']], dtype=object)

## 

## 4. description
After appending the content data to the description on places where the two differ. We will now use NLP.

In [120]:
# Applying the NLP function to the 'description' column
dataset["description"] = dataset["description"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["description"].values)
print(dataset[dataset['id'] == 234]["description"].values)

['follow compani project present first group task entail select project form team phase facilit familiar compani project initi includ requir gather plan team build project primarili emphas gener engin skill cover product qualiti design rd environment aspect custom support less emphasi special skill like mechan electron code']
['tijden modul ler welk kennis vaardig nodig hebt ondernemerschap personal branding werk presenter netwerk opbouw belastingaangift subsidies aanvrag']


## 5. location
Location column is now of datatype string. We will change this to be an array. We do this because sometimes a string is used which says two locations. This is hard to use for filtering, better if its two string of both locations inside of an array. E.g.: "Tilburg & Breda" --> ["Tilburg", "Breda"]

In [121]:
# Function to get all locations inside an array and for the two 'special cases' to be two string inside of the array representing both locations
def normalize_location(value):
    if pd.isna(value):
        return []

    text = str(value).strip()

    # Special cases
    if text == "Breda en Den Bosch":
        return ["Breda", "Den Bosch"]
    if text == "Den Bosch en Tilburg":
        return ["Den Bosch", "Tilburg"]

    # Default will just convert the single location to be inside of an array for data type consistency
    return [text]

dataset["location"] = dataset["location"].apply(normalize_location)

# Check where both scenarios can be seen
dataset.head(183)

,id,name,description,studycredit,location,contact_id,level,learningoutcomes,Rood,Groen,Blauw,Geel,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,modul ler gedrag jezelf ander kunt begrijp bei...,15,[Den Bosch],58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,4.0,2.0,1.0,5.0,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,learn work abroad,student kiez binn stam opleid verpleegkund ste...,15,[Den Bosch],58,NLQF5,De student toont professioneel gedrag conform ...,5.0,3.0,1.0,1.0,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,proactiev zorgplann,jeroen bosch ziekenhuis grag sam opleid verple...,15,[Den Bosch],59,NLQF5,De student past pro actieve zorgplanning toe b...,NaN,NaN,NaN,NaN,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,rouw verlies,modul stil gestan rouw verlies vanuit diver in...,30,[Den Bosch],58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,acuut complex zorg,modul student verdiep acut complex zorg binn v...,30,[Den Bosch],58,NLQF6,De student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,365,minor procesautomatis benno vri,student ler vaardig dusdan student industriel ...,30,[Breda],101,NLQF6,The student learns how to create a business pl...,NaN,NaN,NaN,NaN,"['ontwerpen', 'modelleren', 'instellen']",0.91,317,4,71,2025-09-06
179,366,didactiek ng nt,student mak kennis vak docent hierbij richt wi...,15,[Breda],101,NLQF6,"nog te bepalen, bv; student bereid een les of ...",NaN,NaN,NaN,NaN,"['didactiek', 'docentschap', 'natuur', 'techni...",0.39,153,3,76,2025-12-27
180,367,proefdierkund,lab proefdier mog werk biomedisch onderzoeker ...,15,[Breda],101,NLQF5,"De student levert, op basis van zelf opgesteld...",NaN,NaN,NaN,NaN,[],0.52,278,2,57,2025-12-22
181,368,modul forensisch onderzoek,aanleid mock up plat delict student forensisch...,15,[Breda],101,NLQF5,nog te bepalen,NaN,NaN,NaN,NaN,"['plaats', 'delict', 'scenario', 'denken', 'fo...",0.37,379,2,52,2025-12-03


## 6. learningoutcomes
Here we will also use NLP to prepare the data for our model later. However, we also found some inconsistenies inside of the data of this column. You can find values such as: 'ntb', 'nog te bepalen, nan, etc. These values will all have to be properly set to NaN values so that our model later does not take 'nog te bepalen' as an input.

First we will remove the inconsistensies, after we will use NLP.

In [122]:
# What we need to remove/set to np.nan inside of the learningoutcomes column
error_values_learningoutcomes_contains = ["ntb", "nog niet bekend", "nog te formuleren", "nog nader te bepalen", "nader te bepalen", "nog te bepalen", "n.n.b."]
error_values_learningoutcomes_specific = ["volgt", "nan"]

# Setting all text to lower casing
dataset["learningoutcomes"] = dataset["learningoutcomes"].str.lower()

# Looping and setting values to np.nan for contain errors and specific errors
for val in error_values_learningoutcomes_contains:
    # Only match the full phrase anywhere in the text
    dataset.loc[dataset["learningoutcomes"].str.contains(re.escape(val), na=False), "learningoutcomes"] = np.nan

# Now handle the specific list with exact matches
dataset.loc[dataset["learningoutcomes"].isin(error_values_learningoutcomes_specific), "learningoutcomes"] = np.nan

# Checking is succesful --> number 6, 7, 8 have all been converted to NaN succesfully. Before they were all value: "Nader te bepalen". ALso checked some random locations. 
dataset.head(10)


,id,name,description,studycredit,location,contact_id,level,learningoutcomes,Rood,Groen,Blauw,Geel,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,modul ler gedrag jezelf ander kunt begrijp bei...,15,[Den Bosch],58,NLQF5,a. je beantwoordt vragen in een meerkeuze kenn...,4.0,2.0,1.0,5.0,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,learn work abroad,student kiez binn stam opleid verpleegkund ste...,15,[Den Bosch],58,NLQF5,de student toont professioneel gedrag conform ...,5.0,3.0,1.0,1.0,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,proactiev zorgplann,jeroen bosch ziekenhuis grag sam opleid verple...,15,[Den Bosch],59,NLQF5,de student past pro actieve zorgplanning toe b...,NaN,NaN,NaN,NaN,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,rouw verlies,modul stil gestan rouw verlies vanuit diver in...,30,[Den Bosch],58,NLQF6,de student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,acuut complex zorg,modul student verdiep acut complex zorg binn v...,30,[Den Bosch],58,NLQF6,de student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19
5,164,kraam kind jeugd,modul student verdiep kram kind jeugdzorg binn...,30,[Den Bosch],58,NLQF6,de student regisseert en voert (deels) zelfsta...,NaN,NaN,NaN,NaN,"['jeugdzorg', 'neonatologie', 'verloskunde', '...",0.40,493,5,39,2025-12-04
6,165,profiel specifiek modul welzijn samenlev jeugd...,profiel specifiek modul welzijn samenlev jeugd...,15,"[Breda, Den Bosch]",58,NLQF5,NaN,NaN,NaN,NaN,NaN,"['jeugdzorg', 'geestelijke', 'gezondheidzorg',...",0.34,170,5,76,2025-11-14
7,166,happi citi happi peopl design lab,focus grot vraagstuk sdg gestuurd concret plek...,15,[Den Bosch],58,NLQF5,NaN,NaN,NaN,NaN,NaN,"['vitale', 'gemeenschappen', 'duurzaam', 'gebo...",0.86,77,3,37,2025-12-27
8,167,build sustain vital commun,bedenkt ontwerpt sam relevant stakeholder wijk...,30,[Den Bosch],58,NLQF6,NaN,NaN,NaN,NaN,NaN,"['vitale', 'gemeenschappen', 'duurzaam', 'gebo...",0.69,298,4,66,2025-09-16
9,168,ggz agoog,bent stat the art kennis handelingsrepertoir m...,30,[Den Bosch],58,NLQF6,toetsvormen zijn afgeleide van beroepsprestati...,NaN,NaN,NaN,NaN,"['herstel', 'en', 'krachtgericht', 'werk', 'le...",0.76,407,2,60,2025-09-08


Now for the NLP part:

In [123]:
# Applying the NLP function to the 'learningoutcomes' column
dataset["learningoutcomes"] = dataset["learningoutcomes"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["learningoutcomes"].values)
print(dataset[dataset['id'] == 234]["learningoutcomes"].values)

['gain abil work divers interdisciplinari team reallif project within intern context develop comprehens skill set encompass gener engin compet project manag account project progress effect present skill also improv profici english languag']
['student demonstreert person groei zes competenties stat domeinspecifiek interdisciplinair crer experimenter innovatiev wijz meervoud duurzam waardecreatie kom']


## 7. color-coded columns
We drop the colour columns (Green, Blue, Red, Yellow) as they are not relevant for our recommendation system mainly because it is not useful metadata for understanding user preferences or item characteristics and most values in these columns are Not A Number (NaN).

In [124]:
# Dropping all 4 columns
dataset.drop(columns=["Rood"], inplace=True)
dataset.drop(columns=["Groen"], inplace=True)
dataset.drop(columns=["Blauw"], inplace=True)
dataset.drop(columns=["Geel"], inplace=True)

# Check
dataset.head()

,id,name,description,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,modul ler gedrag jezelf ander kunt begrijp bei...,15,[Den Bosch],58,NLQF5,a beantwoordt vrag meerkeuz kennistoet waarin ...,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,learn work abroad,student kiez binn stam opleid verpleegkund ste...,15,[Den Bosch],58,NLQF5,student toont professionel gedrag conform bero...,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,proactiev zorgplann,jeroen bosch ziekenhuis grag sam opleid verple...,15,[Den Bosch],59,NLQF5,student past pro actiev zorgplann toe binn pro...,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,rouw verlies,modul stil gestan rouw verlies vanuit diver in...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,acuut complex zorg,modul student verdiep acut complex zorg binn v...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19


## 8. module_tags
NLP will once again be used here, but first we need to set all of the empty rows ( [], ['ntb'] ) to np.nan. We do this because these rows provide no information and will mess up the model if we don't correct count them out with np.nan.

In [125]:
# Convert string representation of lists to actual lists
dataset["module_tags"] = dataset["module_tags"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now set NaN for empty lists or ['ntb']
dataset.loc[dataset["module_tags"].apply(lambda x: x == [] or x == ['ntb']), "module_tags"] = np.nan

# this position was ['ntb'] before, now we can check to see if it's fixed. Ofcourse we also check on several other locations and other cases like []
dataset.iloc[20]

id                                                                     179
name                                                         proactiv nurs
description              ontwikkel diepgaand begrip klinisch redener on...
studycredit                                                             15
location                                                           [Breda]
contact_id                                                              63
level                                                                NLQF5
learningoutcomes                                                       NaN
module_tags                                                            NaN
interests_match_score                                                  0.7
popularity_score                                                       185
estimated_difficulty                                                     2
available_spots                                                         42
start_date               

After properly handling the empty data rows. We now have to get the single string outside of their array and append them into 1 big string so that we can use our NLP function.

In [126]:
dataset["module_tags"] = dataset["module_tags"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else x
)
dataset["module_tags"].head()

0    brein gedragsbeinvloeding ontwikkelingspsychol...
1    internationaal persoonlijke ontwikkeling verpl...
2         proactieve zorgplanning cocreatie ziekenhuis
3    rouw verlies palliatieve zorg redeneren trauma...
4       acute zorg complexiteit ziekenhuis revalidatie
Name: module_tags, dtype: object

Now for the NLP part:

In [127]:
# Applying the NLP function to the 'module_tags' column
dataset["module_tags"] = dataset["module_tags"].apply(NLP)

# Check to see if both languages are processed correctly 
print(dataset[dataset['id'] == 315]["module_tags"].values)
print(dataset[dataset['id'] == 234]["module_tags"].values)

['european reallif project']
['person ontwikkel interdisciplinair innovatie prototyp valorisatie']


## 9. popularity_score
The scores of this column range from 0-500. We will normalize them on a scale from 0-1. 

In [128]:
# Divide scores by max score
dataset["popularity_score"] = dataset["popularity_score"] / 500

# Check
dataset.head()


,id,name,description,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,modul ler gedrag jezelf ander kunt begrijp bei...,15,[Den Bosch],58,NLQF5,a beantwoordt vrag meerkeuz kennistoet waarin ...,brein gedragsbeinvloed ontwikkelingspsychologi...,0.54,0.638,1,79,2025-12-24
1,160,learn work abroad,student kiez binn stam opleid verpleegkund ste...,15,[Den Bosch],58,NLQF5,student toont professionel gedrag conform bero...,international person ontwikkel verpleegkund,0.92,0.344,5,56,2025-12-20
2,161,proactiev zorgplann,jeroen bosch ziekenhuis grag sam opleid verple...,15,[Den Bosch],59,NLQF5,student past pro actiev zorgplann toe binn pro...,proactiev zorgplann cocreatie ziekenhuis,0.78,0.434,5,55,2025-09-23
3,162,rouw verlies,modul stil gestan rouw verlies vanuit diver in...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,rouw verlies palliatiev zorg redener trauma cu...,0.69,0.908,1,54,2025-10-25
4,163,acuut complex zorg,modul student verdiep acut complex zorg binn v...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,acut zorg complexiteit ziekenhuis revalidatie,0.40,0.356,5,38,2025-11-19


## 10. Finalizing

We have completed the data cleaning process for our dataset. The cleaned data is now ready for modeling. We will save the cleaned dataset as a new CSV file to ensure that we can easily access and use it in future steps of our project.

In [129]:
dataset.to_csv(Path("../Data/Cleaned/cleaned_dataset.csv"), index=False)

This is the final structure and data of our cleaned dataset:

In [130]:
dataset.head()

,id,name,description,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismak psychologi,modul ler gedrag jezelf ander kunt begrijp bei...,15,[Den Bosch],58,NLQF5,a beantwoordt vrag meerkeuz kennistoet waarin ...,brein gedragsbeinvloed ontwikkelingspsychologi...,0.54,0.638,1,79,2025-12-24
1,160,learn work abroad,student kiez binn stam opleid verpleegkund ste...,15,[Den Bosch],58,NLQF5,student toont professionel gedrag conform bero...,international person ontwikkel verpleegkund,0.92,0.344,5,56,2025-12-20
2,161,proactiev zorgplann,jeroen bosch ziekenhuis grag sam opleid verple...,15,[Den Bosch],59,NLQF5,student past pro actiev zorgplann toe binn pro...,proactiev zorgplann cocreatie ziekenhuis,0.78,0.434,5,55,2025-09-23
3,162,rouw verlies,modul stil gestan rouw verlies vanuit diver in...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,rouw verlies palliatiev zorg redener trauma cu...,0.69,0.908,1,54,2025-10-25
4,163,acuut complex zorg,modul student verdiep acut complex zorg binn v...,30,[Den Bosch],58,NLQF6,student regisseert voert del zelfstand samenwe...,acut zorg complexiteit ziekenhuis revalidatie,0.40,0.356,5,38,2025-11-19
